In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")

from datetime import datetime

import altair as alt
import branca
import branca.colormap as cm
import calitp_data_analysis.magics
import geopandas as gpd
import google.auth
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from shared_utils import webmap_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("new_readable.yml")
credentials, project = google.auth.default()

import _new_operator_report_utils as utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
def formatted(number):
    return "{:,}".format(number)

In [4]:
analysis_name = 'Yolo County Transportation District'

In [5]:
#  %%capture_parameters
#  analysis_name

In [6]:
# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [7]:
date = datetime.today().strftime("%Y_%m")

In [8]:
# Load datasets
schedule_rt_route_direction_summary = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/schedule_rt_route_direction_summary_{date}.parquet"
operator_hourly_summary = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_operator_hourly_summary_{date}.parquet"
fct_monthly_routes = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_monthly_routes_{date}.parquet"

In [9]:
schedule_rt_route_direction_summary_df = pd.read_parquet(schedule_rt_route_direction_summary,filters=[[("Analysis Name", "==", analysis_name)]])

In [10]:
operator_hourly_summary_df = pd.read_parquet(operator_hourly_summary,filters=[[("Analysis Name", "==", analysis_name)]])

In [23]:
fct_monthly_route_df = gpd.read_parquet(fct_monthly_routes,filters=[[("Analysis Name", "==", analysis_name)]], storage_options={"token": credentials.token}).reset_index()

In [24]:
fct_monthly_route_df['Number'] = fct_monthly_route_df.index

In [14]:
analysis_name_edited = analysis_name.replace(" ","_").lower()
operator_webmap_file = f"{analysis_name_edited}_routes"

# {analysis_name}

## Operator Overview

## Service Area

In [18]:
color_map = cm.linear.Spectral_11.scale(0, len(fct_monthly_route_df))

In [25]:
try:
    operator_map = webmap_utils.set_state_export(
    fct_monthly_route_df.drop_duplicates(subset = ["Route Name"])[["Route Name","Geometry", "Number"]],
    subfolder = "operator_digest/",
    filename=operator_webmap_file,
    map_title=f"Transit Routes for {analysis_name}",
    cmap=color_map,
    color_col="Number",
    overwrite = True
    )
    webmap_utils.render_spa_link(operator_map["spa_link"], text=f"Routes for {analysis_name}")
    webmap_utils.display_spa_map(operator_map["spa_link"])
except:
    display(Markdown(f"""{analysis_name} doesn't have an route geographies."""))

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJUcmFuc2l0IFJvdXRlcyBmb3IgWW9sbyBDb3VudHkgVHJhbnNwb3J0YXRpb24gRGlzdHJpY3QiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL29wZXJhdG9yX2RpZ2VzdC95b2xvX2NvdW50eV90cmFuc3BvcnRhdGlvbl9kaXN0cmljdF9yb3V0ZXMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszOC42MDA5MDg1NDg4MjQzODUsIC0xMjEuNjQwMzc1NjMyNDg2ODddLCAiem9vbSI6IDEzfQ==" target="_blank">Open Routes for Yolo County Transportation District in New Tab</a>

## Service Hours 

In [28]:
utils.create_hourly_summary(operator_hourly_summary_df, "Weekday")

alt.LayerChart(...)

In [29]:
utils.create_hourly_summary(operator_hourly_summary_df, "Saturday")

alt.LayerChart(...)

In [30]:
utils.create_hourly_summary(operator_hourly_summary_df, "Sunday")

alt.LayerChart(...)

## Detailed Route Overview

In [31]:
utils.create_scheduled_minutes(schedule_rt_route_direction_summary_df)

alt.HConcatChart(...)

In [32]:
utils.create_scheduled_trips(schedule_rt_route_direction_summary_df)

alt.HConcatChart(...)

In [33]:
utils.create_frequency(schedule_rt_route_direction_summary_df)

alt.HConcatChart(...)

In [34]:
utils.create_text_graph(schedule_rt_route_direction_summary_df)

alt.Chart(...)